so I will have data as ndarray at bottleneck as N by D
I will have the class labels acquired by kmeans
I want to find correspondance samples as
a) two random samples that are in cluster
b) some sample and the center sample

which wil be acquired as in and out indices by a function that takes nxd array and cluster labels

In [114]:
import importlib as impL
import sys
sys.path.insert(1,'/home/doga/GithUBuntU/keyhandshapediscovery/vae_torch')
sys.path.insert(1,'/home/doga/GithUBuntU/keyhandshapediscovery')
import vae_torch_model as vtm
import helperFuncs as funcH
import numpy as np

In [207]:
def load_from_saved_corr_file(fn = '/home/doga/Desktop/correspondance_find.npz'):
    a = np.load(fn)
    predictions = a['predictions']
    bottleneck_vec = a['bottleneck_vec']
    labels = a['labels']
    print("loaded from file - ", fn)
    print(predictions.shape)
    print(bottleneck_vec.shape)
    print(labels.shape)
    return bottleneck_vec, predictions, labels

In [209]:
# np.savez('/home/doga/Desktop/correspondance_find.npz', bottleneck_vec=bottleneck_vec, predictions=pred_vec, labels=lab_vec)
bottleneck_vec, predictions, labels = load_from_saved_corr_file('/home/doga/Desktop/correspondance_find_2.npz')

loaded from file -  /home/doga/Desktop/correspondance_find_2.npz
(60000,)
(60000, 32)
(60000,)


In [112]:
b_v = np.random.rand(10,3)
print(b_v)

[[0.62174144 0.68211817 0.24458723]
 [0.50007153 0.74286471 0.06219046]
 [0.36406782 0.67564379 0.02153782]
 [0.12785142 0.50176279 0.51418295]
 [0.24824411 0.56888874 0.05086619]
 [0.83598948 0.19389137 0.74928321]
 [0.12616682 0.82114396 0.90132345]
 [0.28343164 0.07044607 0.223009  ]
 [0.07490857 0.91460312 0.81610949]
 [0.47475892 0.08782649 0.93664046]]


In [178]:
impL.reload(funcH)
corr_inds, centroid_df = funcH.get_cluster_correspondance_ids(b_v, cluster_ids=[0,0,0,0,1,1,2,2,1,1], correspondance_type="knear4", verbose=0)
print(pd_df({"in":corr_inds[0],"out":corr_inds[1]}))
print(centroid_df)

    in  out
0    0    1
1    4    1
2    5    2
3    5    1
4    3    1
5    4    3
6    0    3
7    1    2
8    2    2
9    3    3
10   2    1
11   3    2
12   8    0
13   5    3
14   1    1
15   7    1
16   8    1
17   8    3
18   5    0
19   8    2
20   4    0
21   1    3
22   3    0
23   9    2
24   9    1
25   0    0
26   9    0
27   1    0
28   6    0
29   7    0
30   9    3
31   0    2
32   2    0
33   6    1
34   4    2
35   2    3
   klusterID  sampleID  distanceEuc  num_of_samples
0          0         2     0.194758               4
1          1         9     0.467323               4
2          2         6     0.511955               2


In [150]:
a,  centroid_df = get_cluster_correspondance_ids(bottleneck_vec, predictions, correspondance_type="shuffle")

In [47]:
import vae_scripts as vs

In [ ]:
vs.run_compare_list(experiments_folder="/home/doga/DataFolder/vaesae_experiments_cor",
                     data_log_keys=['tr_te', 'te'],
                     loss_key_list=[ 'bottleneck_kmeans', 'reconstruction'],
                     exp_base_name='exp_linear_vae_FMNIST_is28_cf',
                     ae_f_name_base='ae_ft_linear_vae_FMNIST_is28.npy')

In [ ]:
import vae_utils as vu
impL.reload(vu)
vu.plot_cf(cf_int=508, data_log_keys = ['tr_te', 'te'], max_act_ep=55, plot_cnt = 3,
            select_id_type='linspace', k_loss_disp_list={'bottleneck_kmeans'},
            experiments_folder='/home/doga/DataFolder/vaesae_experiments_cor',
            exp_base_name='exp_linear_vae_FMNIST_is28_cf', ae_f_name_base='ae_ft_linear_vae_FMNIST_is28.npy')

tr_te  - log is loaded with len:  33
te  - log is loaded with len:  33
skipping -  valid loss
skipping -  reconstruction
skipping -  sparsity
bottleneck_kmeans :
skipping -  bottleneck_act
0*/home/doga/DataFolder/vaesae_experiments_cor/exp_linear_vae_FMNIST_is28_cf508/btl_000_tr_va_.png*
0*/home/doga/DataFolder/vaesae_experiments_cor/exp_linear_vae_FMNIST_is28_cf508/btl_000_tr_te_.png*
1*/home/doga/DataFolder/vaesae_experiments_cor/exp_linear_vae_FMNIST_is28_cf508/btl_016_tr_va_.png*
1*/home/doga/DataFolder/vaesae_experiments_cor/exp_linear_vae_FMNIST_is28_cf508/btl_016_tr_te_.png*
2*/home/doga/DataFolder/vaesae_experiments_cor/exp_linear_vae_FMNIST_is28_cf508/btl_032_tr_va_.png*
2*/home/doga/DataFolder/vaesae_experiments_cor/exp_linear_vae_FMNIST_is28_cf508/btl_032_tr_te_.png*


In [148]:
get_cluster_correspondance_ids(b_v, cluster_ids=[0,0,0,0,1,1,2,2,1,1], correspondance_type="knear3", verbose=1)

(45,)
(10, 10)


IndexError: index 4 is out of bounds for axis 0 with size 4

In [212]:
bottleneck_vec, pred_vec, labels = load_from_saved_corr_file('/home/doga/Desktop/correspondance_find_2.npz')

loaded from file -  /home/doga/Desktop/correspondance_find_2.npz
(60000,)
(60000, 32)
(60000,)


In [235]:
bottleneck_vec = b_v
pred_vec=np.asarray([0,0,0,0,1,1,2,2,1,1],dtype=int)
labels=np.asarray([2,2,2,1,0,0,0,3,3,3],dtype=int)

In [240]:
correspondance_tuple

(array([2, 9, 2, 7, 3, 3, 4, 0, 6, 8, 1, 5, 7, 5, 9, 4, 0, 6, 1, 8]),
 array([0, 1, 1, 1, 2, 0, 2, 2, 0, 0, 0, 0, 0, 3, 2, 1, 1, 1, 2, 3]))

In [247]:
correspondance_type = 'knear2'
impL.reload(funcH)
correspondance_tuple, centroid_df = funcH.get_cluster_correspondance_ids(X=bottleneck_vec, cluster_ids=pred_vec,
                                                             correspondance_type=correspondance_type,
                                                             verbose=5)


***
knear-row0
 [0 1 2 3]
cluster_inds:
 [0 1 2 3]
d_inds in: [[1 2]
 [2 0]
 [1 0]
 [2 0]]
i =  0
sidx = 
 [0 0 1 1 2 2 3 3]
d_inds = 
 [1 2 2 0 1 0 2 0]
iin_cur.shape(8,), out_cur.shape(8,)
iin= [0 0 1 1 2 2 3 3]
out= [1 2 2 0 1 0 2 0]

***
knear-row1
 [4 5 8 9]
cluster_inds:
 [4 5 8 9]
d_inds in: [[2 1]
 [3 0]
 [0 3]
 [1 2]]
i =  1
sidx = 
 [4 4 5 5 8 8 9 9]
d_inds = 
 [8 5 9 4 4 9 5 8]
iin_cur.shape(8,), out_cur.shape(8,)
iin= [4 4 5 5 8 8 9 9]
out= [8 5 9 4 4 9 5 8]

***
knear-row2
 [6 7]
cluster_inds:
 [6 7]
d_inds in: [[1 0]
 [0 1]]
i =  2
sidx = 
 [6 6 7 7]
d_inds = 
 [7 6 6 7]
iin_cur.shape(4,), out_cur.shape(4,)
iin= [6 6 7 7]
out= [7 6 6 7]
shuffle all
inds_in.shape(20,), inds_out.shape(20,)


In [248]:
from sklearn.metrics import confusion_matrix
impL.reload(funcH)


In [249]:
analyze_corresondance_results(correspondance_tuple, centroid_df, pred_vec, labels)

correspondance results:
        labels  sampleCounts
labels                      
0            0             2
2            2             4
3            3             4
_confMat_corr_preds - acc(100.0000)
confMat - acc(60.0000), correspondance match:
    0  1  2  3
0  3  0  0  3
1  0  0  2  0
2  0  0  6  0
3  3  0  0  3


In [243]:
d_inds = np.asarray([2,1,3,0,0,3,1,2], dtype=int)
cluster_inds = np.asarray([4,5,8,9], dtype=int)

In [244]:
d_inds

array([2, 1, 3, 0, 0, 3, 1, 2])

In [245]:
cluster_inds

array([4, 5, 8, 9])

array([8, 5, 9, 4, 4, 9, 5, 8])